# Electric vehicle credit

The US government offers a non-refundable tax credit of up to \$7,500 for each new electric vehicle purchased.

## Examples

Consider a filer in Massachusetts who buys a new Ford Mustang Mach-E---the [top-selling electric vehicle](https://www.kbb.com/best-cars/most-popular-electric-cars/) eligible for the credit as of July 2022 (Teslas are no longer eligible)---in 2022.
The [2021 Mach-E's battery starts at 70 kilowatt hours of capacity](https://media.ford.com/content/dam/fordmedia/North%20America/US/product/2022/mach-e/pdf/22MACH_E_Tech_Specs.pdf), entitling it to the [full \$7,500 credit](https://www.irs.gov/businesses/irc-30d-new-qualified-plug-in-electric-drive-motor-vehicle-credit).

However, because the credit is non-refundable, the filer will only receive a subsidy if they have income.
A single filer has to earn \$13,000 to receive any benefit from the credit, and about \$67,000 to receive the full \$7,500.
Married joint filers and those with children need higher incomes to benefit from the credit.


In [2]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {"0": GRAY, "1": LIGHT_BLUE, "2": BLUE, "3": DARK_BLUE}


def make(adults, children, ev_battery_capacity):
    sim = IndividualSim(year=2022)
    # Specify age 25 to qualify for EITC.
    sim.add_person(name="head", age=25, is_tax_unit_head=True)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse")
        members.append("spouse")
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=5)
        members.append(child)
    sim.add_tax_unit(
        name="tax_unit",
        members=members,
        purchased_qualifying_new_electric_vehicle=True,
        new_electric_vehicle_battery_capacity=ev_battery_capacity,
        premium_tax_credit=0,
    )
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MA")
    sim.vary("employment_income", max=120_000)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            spm_unit_net_income=sim.calc("spm_unit_net_income")[0],
            credit=sim.calc("new_electric_vehicle_credit")[0],
            adults=adults,
            children=str(children),  # String for color mapping.
            ev_battery_capacity=ev_battery_capacity,
        )
    )


l = []
for adults in [1, 2]:
    for children in range(0, 4):
        for ev_battery_capacity in [0, 70]:
            l.append(make(adults, children, ev_battery_capacity))

df = pd.concat(l)

wide = df.pivot(
    index=["employment_income", "adults", "children"],
    columns="ev_battery_capacity",
    values="spm_unit_net_income",
).reset_index()
wide["benefit"] = wide[70] - wide[0]

LABELS = dict(
    employment_income="Employment income",
    benefit="Net electric vehicle credit",
    children="Children",
    adults="Adults",
)

fig = px.line(
    wide,
    "employment_income",
    "benefit",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Net benefit from electric vehicle credit",
    color_discrete_map=COLOR_MAP,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor=LIGHT_GRAY,
    yaxis_gridcolor=LIGHT_GRAY,
)
fig.show()